# LAMsec Benchmark Notebook
Run the full attack dataset, compute metrics, and summarize results.

## 1. Setup (Colab optional)

In [ ]:
# Colab install (uncomment if needed)
# !pip install -q transformers sentence-transformers rapidfuzz gradio pyyaml torch==2.2.0 --extra-index-url https://download.pytorch.org/whl/cpu

## 2. Imports

In [ ]:
import json, time, yaml, statistics as stats
from pathlib import Path
from src.benchmark_runner import run_all
from src.metrics import ads, svi, gcr
from src.utils import set_deterministic
set_deterministic(42)
print('Ready.')

## 3. Execute Benchmark

In [ ]:
start = time.time()
results = run_all()
elapsed = round(time.time() - start, 2)
len(results), elapsed

## 4. Aggregate Metrics

In [ ]:
ads_vals = [r['ads'] for r in results]
gcr_vals = [r['gcr'] for r in results]
svi_vals = [r['svi'] for r in results]
succ = [r['attack_success'] for r in results]
summary = {
    'cases': len(results),
    'avg_ads': round(sum(ads_vals)/len(ads_vals),4),
    'median_ads': stats.median(ads_vals),
    'avg_gcr': round(sum(gcr_vals)/len(gcr_vals),4),
    'attack_success_rate': round(sum(1 for x in succ if x)/len(succ),4),
    'svi_positive_rate': round(sum(1 for v in svi_vals if v>0)/len(svi_vals),4),
} 
summary

## 5. Sample Rows

In [ ]:
for r in results[:3]:
    print(r['id'], r['category'], r['ads'], r['svi'], r['gcr'], r['attack_success'])

## 6. Persist Results

In [ ]:
out_path = Path('results/benchmark_notebook.json')
out_path.parent.mkdir(exist_ok=True)
out_path.write_text(json.dumps(results, indent=2))
summary_path = Path('results/summary_notebook.json')
summary_path.write_text(json.dumps(summary, indent=2))
out_path, summary_path

## 7. (Optional) Simple Distribution Plots

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(6,3)); plt.hist([r['ads'] for r in results], bins=15, color='steelblue'); plt.title('ADS Distribution'); plt.show()
plt.figure(figsize=(6,3)); plt.hist([r['gcr'] for r in results], bins=15, color='seagreen'); plt.title('GCR Distribution'); plt.show()
plt.figure(figsize=(6,3)); plt.hist([r['svi'] for r in results], bins=10, color='indianred'); plt.title('SVI Distribution'); plt.show()